# **Main Goal: Create an CNN that can detect "hot spots" from thermal images**

# New Section

# **Get The Data**

In [ ]:
#Import The Essentials

#Import Modules
import tensorflow as tf
import numpy as np
from tensorflow import keras

#Import Pillow Image Tools
import PIL
import PIL.Image

#Display Version of TensorFlow
print("TensorFlow Version:", tf.__version__)

#Print Number of GPUs being Used
print("The Number of GPUs Available is: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow Version: 2.9.2
The Number of GPUs Available is:  1


In [ ]:
#Load The Dataset

#Allow Google Colab to Access Files
from google.colab import files

#Names Dataset "data"
data = files.upload()

#Confirms Upload
for fn in data.keys():
  print('\nUser uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(data[fn])))

#Unzip Data
from zipfile import ZipFile

with ZipFile('cnn_dataset.zip', 'r') as unzipped_data:
  unzipped_data.extractall('/content/data')

print('\nThe data was successfully unzipped! See the unzipped data below:')

#Confirm that Data was Unzipped
!ls '/content/data'

#Message If Folder Isn't Showing:
print('\nIf you cannot see the folder in the directory list, hit the directory refresh button (middle button)')

#Delete the Original .zip File
!rm cnn_dataset.zip
print('\nThe original zip file has been deleted to free up space within the environment.')

FileNotFoundError: ignored

### Source for the following cells: https://www.tensorflow.org/tutorials/load_data/images

In [ ]:
#Get the Dataset Working
import pathlib
dataset = '/content/data/cnn_dataset'
fire_data = pathlib.Path(dataset)

In [ ]:
#Figure Out Number of Images in Dataset
image_count = len(list(fire_data.glob('*/*.jpg')))
print(image_count)

In [ ]:
#Create A Dataset

#Define Parameters
batch_size = 10000
img_height = 64
img_width = 64

In [ ]:
#Create the Training Dataset
training_set = tf.keras.utils.image_dataset_from_directory(
    fire_data,
    validation_split = 0.25,
    subset = 'training',
    seed = 123,
    image_size = (img_height, img_width), 
    batch_size=batch_size
)

In [ ]:
#Create the Testing Dataset
testing_set = tf.keras.utils.image_dataset_from_directory(
    fire_data,
    validation_split = 0.25,
    subset = 'validation',
    seed = 123,
    image_size = (img_height, img_width), 
    batch_size=batch_size
)

In [ ]:
#Create The Class Names
class_names = training_set.class_names
print(class_names)

In [ ]:
#Import MatPlotLib
import matplotlib.pyplot as plt

#Plot First 9 Images from Training Dataset
plt.figure(figsize=(10, 10))
for images, labels in training_set.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])

# Convert Training Set To NumPy Arrays

In [ ]:
#Convert To NumPy Arrays (Training Set)
for image_batch, labels_batch in training_set:

  #Convert the Training Dataset into NumPy Arrays
  x_train = np.array(image_batch)
  y_train = np.array(labels_batch)

  #Confirm the Datatype and Shapes of "x_train"
  print("x_train:")
  print("Image Class Type:", type(x_train))
  print("Images Shape:", x_train.shape)

  #Confirm the Datatype and Shapes of "y_train"
  print("\ny_train:")
  print("Labels Class Type:", type(y_train))
  print("Images Shape:", y_train.shape)

In [ ]:
#Changing the Image Type from "unit8" to "float32"
x_train = x_train.astype('float32')/255

#Changing the Label Type from "unit8" to "int64"
y_train = y_train.astype('int64')

In [ ]:
#Display First 9 Images of Training Set (To Verify Image and Label Match-Up)

#Change Size of Images in Plot
plt.figure(figsize=(10, 10))

#For Loop to Plot Images
for i in range(9):  
  
  #Define Subplot
  plt.subplot(3, 3, i+1)

  #Plot Raw Image Pixel Data
  plt.imshow(x_train[i])

  #Display Labels on Images
  plt.title(class_names[y_train[i]])

#Show Images
plt.show()

# Convert the Testing Set to NumPy Arrays

In [ ]:
#Convert To NumPy Arrays (Testing Set)
for image_batch, labels_batch in testing_set:
  
  #Convert Testing Set into NumPy Arrays
  x_test = np.array(image_batch)
  y_test = np.array(labels_batch)

  #Confirm the Datatype and Shapes of "x_test"
  print("x_test:")
  print("Image Class Type:", type(x_test))
  print("Images Shape:", x_test.shape)

  #Confirm the Datatype and Shapes of "y_test"
  print("\ny_test:")
  print("Label Class Type:", type(y_test))
  print("Label Shape:", y_test.shape)

In [ ]:
#Changing the Image Type from "unit8" to "float32"
x_test = x_test.astype('float32')/255

#Changing the Label Type from "unit8" to "int64"
y_test = y_test.astype('int64')

In [ ]:
#Display First 9 Images of Testing Set (To Verify Image and Label Match-Up)

#Change Size of Images in Plot
plt.figure(figsize=(10, 10))

#For Loop to Plot Images
for i in range(9):  
  
  #Define Subplot
  plt.subplot(3, 3, i+1)

  #Plot Raw Image Pixel Data
  plt.imshow(x_test[i])

  #Display Labels on Images
  plt.title(class_names[y_test[i]])

#Show Images
plt.show()

# Grayscale The Images of the Whole Dataset

In [ ]:
#Grayscale Training and Testing Sets
grayscale_x_train = tf.image.rgb_to_grayscale(x_train[:])
grayscale_x_test = tf.image.rgb_to_grayscale(x_test[:])

#Convert to NumPy Arrays and Check Shapes
grayscale_x_train = grayscale_x_train.numpy()
print('The Shape of the NumPy Training Set is: ', grayscale_x_train.shape)

grayscale_x_test = grayscale_x_test.numpy()
print('The Shape of the NumPy Testing Set is: ', grayscale_x_test.shape)

#Display the First Image of the Training Set
gray_image = tf.image.rgb_to_grayscale(x_train[1])
plt.imshow (tf.squeeze(gray_image), cmap = 'gray')

#The Model

In [ ]:
#Convolutional Neural Network
cnn = tf.keras.Sequential([
      
    #Convolutional Layer #1
    tf.keras.layers.Conv2D(filters = 64, kernel_size = (6, 6), strides = (1, 1), activation = 'relu', padding = 'same', input_shape = (64, 64, 1)),

    #Pooling Layer #1
    tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),

    #Convolutional Layer #2
    tf.keras.layers.Conv2D(filters = 32, kernel_size = (2, 2), activation = 'relu'),

    #Pooling Layer #2
    tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),

    #Flatten Output
    tf.keras.layers.Flatten(),

    #Dense Layer #1
    tf.keras.layers.Dense(128, activation = 'relu'),

    #Dense Layer #2
    tf.keras.layers.Dense(64, activation = 'relu'),

    #Dense Layer #3
    tf.keras.layers.Dense(32, activation = 'relu'),

    #Dense Layer #4
    tf.keras.layers.Dense(16, activation = 'relu'),

    #Dense Layer #5
    tf.keras.layers.Dense(8, activation = 'relu'),

    #Dense Layer #6
    tf.keras.layers.Dense(4, activation = 'softmax'),

])

In [ ]:
#Compile The Model
cnn.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
#Model Summary
cnn.summary()

In [ ]:
#Train The Model
EPOCHS = 5
history = cnn.fit(
    grayscale_x_train, y_train,
    batch_size = 12, epochs = EPOCHS, validation_split = 0.1, verbose = 1,
)

In [ ]:
#Evaluate The Model
test_loss, test_acc = cnn.evaluate(grayscale_x_test, y_test, verbose = 1)
print('The Test Set Loss is: {0:0.4f} and the Test Set Accuracy is: {1:0.4}%'.format(test_loss, 100*test_acc))

In [ ]:
#Import MatPlotLib
import matplotlib.pyplot as plt

#Define Variables
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

#Plot First Graph
plt.figure(figsize = (8, 8))
plt.subplot(2, 2, 1)
plt.plot(acc, label = "Training Accuracy")
plt.plot(val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()), 1])
plt.title('Training and Validation Accuracy')

#Plot the Second Graph
plt.subplot(2, 1, 2)
plt.plot(loss, label = 'Training Loss')
plt.plot(val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0, 1.0])
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
#Make Prediction
prediction = cnn.predict(grayscale_x_test[500].reshape(1, 64, 64, 1))
np_predict = np.argmax(prediction)
print("The model predicts the following image to be a member of Class {}".format(np_predict) + " which translates to {}".format(class_names[np_predict]) + '.')

#Show Image
plt.imshow(grayscale_x_test[500].reshape(64, 64))
plt.title(class_names[np_predict])

#Code To Deploy Model Files - https://www.analyticsvidhya.com/blog/2021/04/easily-deploy-your-machine-learning-model-into-a-web-app-netlify/#h2_3

In [ ]:
#Save model into a variable and save model
cnn_deployed = "./cnn.h5"
cnn.save(cnn_deployed)

In [ ]:
#Import TensorFlow JS
!pip install tensorflowjs

In [ ]:
#Convert using TensorFlowjs
!tensorflowjs_converter --input_format=keras '/content/cnn.h5' ./